In [55]:
# ECDSA with Threshold Signatures

# We import the necessary libraries and modules
import secrets # Python's secure random number generator
import math # python's math module 
from phe import paillier # Python's Paillier library
from app.ecdsa import * # ECDSA module from the app package


# This function is a representation of a public broadcast channel
def public_broadcast(message):
    print(f"[BROADCAST] {message}")
    
# this function is a representation of sending a message to a peer    
def send_to_peer(i, data):
    print(f"[TO P{i}] {data}")
    
    
# This function is for create a compartition of a secret using Shamir's Secret Sharing Scheme given a secret, t, n and a prime number

def shamir_split(secret, t, n, prime):
    coeffs = [secret] + [secrets.randbelow(prime) for _ in range(t-1)]
    shares = []
    for i in range(1, n+1):
        x = i
        # eval polinomio
        fx = 0
        power = 1
        for c in coeffs:
            fx = (fx + c * power) % prime
            power = (power * x) % prime
        shares.append((i, fx))
    return shares

# This function is for combine the shares and get the secret using Shamir's Secret Sharing Scheme given a list of shares and a the same prime number used in the split function, We need to know the value of t before using this function 
def shamir_combine(shares, prime):
    total = 0
    k = len(shares)
    for i in range(k):
        xi, yi = shares[i]
        num = 1
        den = 1
        for j in range(k):
            if i != j:
                xj, _ = shares[j]
                num = (num * (-xj)) % prime
                den = (den * (xi - xj)) % prime
        # inverso de den
        inv_den = pow(den, prime-2, prime)
        li = (num * inv_den) % prime
        contribution = (yi * li) % prime
        total = (total + contribution) % prime
    return total


# The Class TrustedDealerCoordinator is a representation of the Trusted Dealer or Coordinator in the Threshold ECDSA scheme, this class is in charge of generating the Paillier KeyPair, receive the shares of the participants, decrypt the shares and distribute the shares to the participants, and finally collect the shares and sign the message
class TrustedDealerCoordinator:
    # The constructor of the class, it receives the number of participants, the threshold, and a prime number for the Shamir's Secret Sharing Scheme
    def __init__(self, n, t, prime_for_shamir):
        # The number of participants
        self.n = n
        # the threshold
        self.t = t
        self.prime = prime_for_shamir
        print("[Coordinator] Generating Paillier KeyPair...")
        self.pubkey, self.privkey = paillier.generate_paillier_keypair()
        self.ciphertext_sum = None
        self.secreto = None
        
    # This function is for send de Paillier public key to the participants
    def get_public_key(self):
        return self.pubkey
    
    # This function is for receive the encrypted shares from the participants
    def receive_final_ciphertext_sum(self, ciph_sum):
        self.ciphertext_sum = ciph_sum
        print("[Coordinator] I recived the addition of part x_i.")
    
    # First recive the shares from the participants and homomorphicly add them
    # Decrypt the total shares and distribute the secrets to the participants
    def decrypt_and_distribute_shares(self):
        if self.ciphertext_sum is None:
            print("[Coordinator] nothing to decrypt.")
            return
        x_global = self.privkey.decrypt(self.ciphertext_sum)
        print(f"[Coordinator] x_global = {x_global} (private key total is clear)")
        self.secreto = x_global        
        shares = shamir_split(x_global, self.t, self.n, self.prime)
        for i, (idx, fx) in enumerate(shares, start=1):
            send_to_peer(i, f"Tu share = {fx}")
            participants[i-1].receive_shamir_share(fx)
    
    # This function is for collect the shares and the random ki values from the participants and sign the message
    def collect_shares_and_sign(self, z, chosen_participants):

        shamir_shares = []
        k_total = 0

        for p in chosen_participants:
            # p.pid => x-coordinate, p.share => y-value
            i = p.pid
            f_i = p.share
            k_total = (k_total + p.k_i) % S256Point.N
        # in this case the prime number is the order of the curve Bitcoin
        x_reconstructed = shamir_combine(chosen, S256Point.N)  
        print(x_reconstructed)

        # 3. Calculate r = (k_total * G).x
        G = S256Point(
            0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
            0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
        )
        R = k_total * G
        r = R.x.num 

        # 4. invert de k_total
        k_inv = pow(k_total, S256Point.N - 2, S256Point.N)

        # 5. s = (z + r*x) * k_inv mod N
        s = ((z + r * x_reconstructed) % S256Point.N) * k_inv % S256Point.N

        print(f"[Coordinator] reconstructed x={x_reconstructed} and sign => r={r}, s={s}")
        return (r, s)
 
            
    # Class Participant is a representation of a participant in the Threshold ECDSA scheme, this class is in charge of generating a secret, encrypt the secret, generate a nonce share, and receive the shares from the coordinator
class Participant:
    def __init__(self, pid, paillier_pubkey):
        self.pid = pid
        self.pk = paillier_pubkey
        self.x_i = None  
        self.share = None 

    def create_secret(self):
        self.x_i = secrets.randbits(128)  #the secret is a random number of 128 bits only for example purposes
        print(f"[P{self.pid}] my local secret is = {self.x_i}")
        return self.x_i

    def encrypt_secret(self):
        if self.x_i is None:
            raise ValueError("No secret to encrypt.")
        ciph = self.pk.encrypt(self.x_i)
        return ciph

    def receive_shamir_share(self, s):
        self.share = s
        print(f"[P{self.pid}] recived my secret: {s}")
    
    def generate_nonce_share(self):
        self.k_i = secrets.randbelow(S256Point.N - 1) + 1
        print(f"[P{self.pid}] generated k_i = {self.k_i}")
        return self.k_i


In [56]:
# we select the large prime same as the order of the curve secp256k1 Bitcoin

prime_for_shamir = S256Point.N
# this values are selected before the execution of the scheme
n=10
t=5

# create the coordinator
coordinator = TrustedDealerCoordinator(n, t, prime_for_shamir)

# Post the public key of paillier for the participants
pk = coordinator.get_public_key()
public_broadcast(f"PaillierPublicKey => (n={pk.n}, g={pk.g})")

# The participants generate their secrets and encrypt them
participants = []
for i in range(1, n+1):
    p = Participant(i, pk)
    participants.append(p)

print("\n--- step A:  every participan generate a random number and encrypted---")
ciphertexts = []
for p in participants:
    p.create_secret()
    ciph = p.encrypt_secret()
    ciphertexts.append(ciph)
    public_broadcast(f"[P{p.pid}] Ciphertext = {ciph.ciphertext()}")

print("\n--- Step B: homomorphic additions ---")

if ciphertexts:
    total_enc = ciphertexts[0]
    for c in ciphertexts[1:]:
        total_enc = total_enc + c  
else:
    total_enc = None

print("[AGREGATOR] complet.")
coordinator.receive_final_ciphertext_sum(total_enc)

print("\n--- Step C: decrypt and sharing ---")
coordinator.decrypt_and_distribute_shares()

print("\n--- Step D: simulating the p2p ---")
for p in participants:
    p.receive_shamir_share(p.share)


# In this step the participants have their secrets generated by the Coordinator  

[Coordinator] Generating Paillier KeyPair...
[BROADCAST] PaillierPublicKey => (n=3254271911402877658702897329502491954564005311308581986640945770156342060179631300955523746829432448352988933204390463568064884358890247519777102607371274870464293915497745771223681005301875647437554888255959835963133028501526479786656020915344112164545896027929385123036477700900272447011597717215389963588352535980399555009899902774057065592415701908416031762030872597644700809470911366627093423555730287684195298010028538160391236769343555023278833860605100970926322353249823174416898444678475867612864488155041377936638277512799205865790571413956303566838846164720565044135205051675350399277954698694827297573655271275761588292349603459263134843673140494262047497640821203782319713859899234079525359388996818709669797695422314842078041203231855915380048094221923419960329391130665710728476471780843073914308146815088825801459795520979426250887421944129963787871338604748507605833079865169911525063349992822693572416

In [57]:
# Check the secrets of the participants are valid


# remember the index starts from 0 in python
chosen = [(1, participants[0].share),
           (3, participants[2].share),
           (5, participants[4].share),
           (7, participants[6].share),            
           (9, participants[8].share)]

print(f"Select Shares for check the reconstruction => {chosen}")

recovered_x = shamir_combine(chosen, prime_for_shamir)
# If the value of recovered_x is equal to the secret of the coordinator, then the scheme is correct
print(recovered_x == coordinator.secreto)






Select Shares for check the reconstruction => [(1, 89081259983344960444225596244803211688598292376363429403876455958804491477767), (3, 20480730266364032350053329081241177725870785540822362003885156743162283376790), (5, 82226434564183584000682473465086264262924858566997521097235552000907717387895), (7, 18408238255327544503452436420966317607628413130545234373709874630055691191037), (9, 103026503228531545611498897194102965388117264100082426260738758137022516086880)]
True


In [58]:
# Now we can create a Public Key from the recovered_x value and sign a message like the ECDSA scheme
e = recovered_x
pk_e = PublicKey(e)
print(e)
print(pk_e.point)


1903342743945397681552882292950497832104
S256Point(92917af45c8e7c8e50e3e3a0de17542512f61e40936802200f3dba72674685cd, b5831646e01ad0400ff535932469bd848110b0db5f289a890aefc31be0129ec7)


In [ ]:
chosen_participants = [participants[0], participants[2], participants[4], participants[6], participants[8]]
participants

for cp in chosen_participants:
    cp.generate_nonce_share()
  
    
z = int.from_bytes(hash256(b'my message'), 'big') #HASH

# Now this signature is equivalent an Signature ECDSA

# We create a diferent signature for the same message. In every case the participants generate a new nonce. This is the reason for the signature is different for the same message
r1,s1 = coordinator.collect_shares_and_sign(z, chosen_participants)

for cp in chosen_participants:
    cp.generate_nonce_share()
  
    
r2,s2 = coordinator.collect_shares_and_sign(z, chosen_participants)


[P1] generated k_i = 63283215534289339516473439470582396211053823357400267178307904839653053551738
[P3] generated k_i = 66628823202079675672479932060528308254104791544358870130722706257083125717903
[P5] generated k_i = 81764450040901919480227846110018378348227050426606151909504017378044113770395
[P7] generated k_i = 107014615553172566333445971546152686601741520427593816484722063596969713929482
[P9] generated k_i = 39862139925124930441153769276608795894206330887688893895803846791943033579828
1903342743945397681552882292950497832104
[Coordinator] reconstructed x=1903342743945397681552882292950497832104 and sign => r=51439308716063180650860860863838221702632758480524290218509425341431911080233, s=28721876715232633235994034193709144859067920671026870882787563460502501220297
[P1] generated k_i = 25185099953618271905886551551873211483831175672610239357648140284730537784868
[P3] generated k_i = 71161410537377944460916984024890294445529944852594517187506750441683573946505
[P5] generated k_i = 

In [78]:
# For fast testing we can use the ECDSA module from the app package

# Create a object of the class Signature with random values
threshold_signature1 = Signature(1,1)


# Asign the r,s values from threshold signature for verification
threshold_signature1.r = r1
threshold_signature1.s = s1

# Create a object of the class Signature with random values
threshold_signature2 = Signature(1,1)


# Asign the r,s values from threshold signature for verification
threshold_signature2.r = r2
threshold_signature2.s = s2


# all the protocol works well if the signature is valid
print('Signature 1 threshold is valid:',pk_e.point.verify(z, threshold_signature1))
print('Signature 2 threshold is valid:',pk_e.point.verify(z, threshold_signature2))

Signature 1 threshold is valid: True
Signature 2 threshold is valid: True
